In [3]:
import pandas as pd
import numpy as np
from datetime import date

Tratando as bases

In [4]:
dict_moedas = ['ZAR', 'AFS', 'AUD', 'AUS', 'ARB', 'ARS', 'CAD', 'CAN', 'CLP', 
'CHL', 'CNY', 'CNH', 'DOL', 'EUR', 'EUP', 'GBP', 'GBR', 'JPY', 'JAP', 'MXN', 'MEX',
'NOK', 'NZD', 'NZL', 'RUB', 'SEK', 'CHF', 'SWI', 'TRY', 'TUQ']

In [40]:
df=pd.read_csv('https://raw.githubusercontent.com/iqfc/B3_Scrapper/main/ft_moedas_1.0.csv',delimiter=',',low_memory=False)
df=df.assign(Contr_venc=df.Contract+'_'+df['Contract Months'])

df['Settlement Price']=df['Settlement Price'].str.replace(',','')
df['Settlement Price']=pd.to_numeric(df['Settlement Price'])

df['Financial Volume (R$)']=df['Financial Volume (R$)'].str.replace(',','')
df['Financial Volume (R$)']=pd.to_numeric(df['Financial Volume (R$)'])

dic_map_1={'JAN':1,'FEV':2,'MAR':3,'ABR':4,'MAI':5,'JUN':6,'JUL':7,'AGO':8,'SET':9,'OUT':10,'NOV':11,'DEZ':12}
dic_map_2={'F':1,'G':2,'H':3,'J':4,'K':5,'M':6,'N':7,'Q':8,'U':9,'V':10,'X':11,'Z':12}

def map_contrat_venc(x):

    if len(x.strip())==3:
        
        year=2000+int(x[1:])
        month=int(dic_map_2.get(x[0]))
        day=1

        return pd.Timestamp(year,month,day)
    else:

        year=2000+int(x[3:])
        month=dic_map_1.get(x[:3])
        day=1

        return pd.Timestamp(year,month,day)

df['Contract Months']=df['Contract Months'].apply(map_contrat_venc)

Volume de transações

In [41]:
ft_vol=df.loc[:,['Contract Months','Contract','Financial Volume (R$)','Date']]
ft_vol['Financial Volume (R$)']=pd.to_numeric(ft_vol['Financial Volume (R$)'])
ft_vol.rename(columns={'Financial Volume (R$)': 'Financial_Volume_RS'}, inplace=True)
ft_vol.Date=pd.to_datetime(ft_vol.Date)

Settlement price dos futuros

In [42]:
ft_price=df.loc[:,['Contract Months','Contract','Settlement Price','Date']]
ft_price['Settlement Price']=pd.to_numeric(ft_price['Settlement Price'])
ft_price.Date=pd.to_datetime(ft_vol.Date)

Importando Spot Prices

In [43]:
# spot=pd.read_csv('CSV_moedas_1.csv',delimiter=',').set_index('Unnamed: 0')
spot=pd.read_csv('https://raw.githubusercontent.com/iqfc/B3_Scrapper/main/CSV_moedas_1.csv',delimiter=',')
spot['DOL']=spot.BRL
spot['Unnamed: 0']=pd.to_datetime(spot['Unnamed: 0'])
spot=spot.set_index('Unnamed: 0')
spot.index.name='Date'


Geting only the contracts available in all bases

In [44]:
contracts_available=pd.DataFrame(spot.columns).merge(pd.DataFrame(df.Contract.unique()),how='inner')
contracts_available=list(contracts_available[0])

spot=spot.loc[:,contracts_available]
ft_price=ft_price[ft_price.Contract.isin(contracts_available)]
ft_vol=ft_vol[ft_vol.Contract.isin(contracts_available)]

Backtest Structure Here

In [45]:
#Detalhe importante, para comparar datas usando o pandas agora precisamos colocar tudo no formado do proprio pandas, diferente do
#que faziamos antes no datetime.date, ou apenas o date, assim o startdate e end_date devem estar no formato Timestap


start_date=pd.Timestamp(2006,1,1)
end_date=pd.Timestamp(2022,12,31)

rebal=1

#não precisa de lookback pois visamos pegar a data mais atual


data_analise=start_date-pd.DateOffset(days=4)

#Presiso dar o slice nas 3 bases ftvol, ft price e spot

#O volume será utilizado como filtro de liquidez, utilizaremos apenas contratos com liquidez maior que 0
volume=ft_vol[(ft_vol.Date > data_analise) & (ft_vol.Date < start_date) & (ft_vol.Financial_Volume_RS>0)]
spot=spot[(ft_vol.Date > data_analise) & (ft_vol.Date < start_date) & (ft_vol.Financial_Volume_RS>0)]